## Prepare and read data

In [1]:
import gc
import itertools
from pathlib import Path
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
import lightgbm as lgb

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import TruncatedSVD, LatentDirichletAllocation, NMF

In [2]:
pd.set_option("max_columns", 300)
pd.set_option("max_rows", 500)

In [3]:
train_df = pd.read_csv("../input/train.csv")
test_df = pd.read_csv("../input/test.csv")

In [4]:
def feature_cache(reset=False):
    def _feature_cache(func):
        def wrapper(train_df, test_df, *args):
            func_name = func.__name__
            train_feat_path = Path("../feature") / f"train_{func_name}.pkl"
            test_feat_path = Path("../feature") / f"test_{func_name}.pkl"
            # if feature exists, load feature
            if train_feat_path.exists() and test_feat_path.exists() and not reset:
                train_feats = pd.read_pickle(train_feat_path).reset_index(drop=True)
                test_feats = pd.read_pickle(test_feat_path).reset_index(drop=True)
                train_df = pd.concat([train_df, train_feats], axis=1)
                test_df = pd.concat([test_df, test_feats], axis=1)
            # if not exists, make feature and save as pickle
            else:
                before_cols = train_df.columns.tolist()
                train_df, test_df = func(train_df, test_df, *args)
                after_cols = train_df.columns.tolist()
                new_cols = [c for c in after_cols if c not in before_cols]
                train_feats = train_df[new_cols]
                test_feats = test_df[new_cols]
                train_feats.to_pickle(train_feat_path)
                test_feats.to_pickle(test_feat_path)            
            return train_df, test_df
        return wrapper

    return _feature_cache

## Feature Engineering

In [5]:
def get_importance_cols(use_num=50):
    # basicなモデルのimportanceを読み込み
    importance_df = pd.read_csv("../data/importance/003_importance.csv")
    imp_feats = importance_df["feature"].iloc[:use_num].tolist()
    return imp_feats

In [6]:
def get_multi_cat_cols(train_df):
    tmp = train_df.iloc[:1000]
    multi_cols = []
    for c in train_df.columns:
        sep_num = tmp[c].astype(str).fillna("").str.contains(";").sum()
        if sep_num > 10:
            multi_cols.append(c)
    return multi_cols

In [7]:
def col_contain(col, feats):
    for feat in feats:
        if col in feat:
            return True
    return False

In [8]:
original_cols = train_df.columns.tolist()

In [9]:
multi_cat_cols = get_multi_cat_cols(train_df)

In [10]:
nume_cols = [
    c for c in list(np.setdiff1d(original_cols, multi_cat_cols))
    if c not in ["Salary", "No"] and "float" in train_df[c].dtype.name
]

In [11]:
cat_cols = [c for c in train_df.columns if c not in multi_cat_cols + nume_cols + ["Salary", "No"]]

In [12]:
len(original_cols), len(cat_cols), len(nume_cols), len(multi_cat_cols)

(128, 65, 40, 21)

### Multi-category encoding

In [13]:
all_feats_dict = {}
for c in tqdm(multi_cat_cols):
    binarizer = MultiLabelBinarizer()
    train_multi_srs = train_df[c].map(lambda x: x.split(";") if x is not np.nan else [])
    test_multi_srs = test_df[c].map(lambda x: x.split(";") if x is not np.nan else [])
    train_arr = binarizer.fit_transform(train_multi_srs)
    test_arr = binarizer.transform(test_multi_srs)
    feat_cols = [f"ohe_{c}_{val}" for val in binarizer.classes_]
    train_feat_df = pd.DataFrame(train_arr, columns=feat_cols, dtype=np.int8)
    test_feat_df = pd.DataFrame(test_arr, columns=feat_cols, dtype=np.int8)
    all_feat_df = pd.concat([train_feat_df, test_feat_df], axis=0, ignore_index=True)
    train_feat_df[f"sum_answer_{c}"] = (train_df[c].str.count(";") + 1).fillna(-1).astype(np.int8)
    test_feat_df[f"sum_answer_{c}"] = (test_df[c].str.count(";") + 1).fillna(-1).astype(np.int8)
    train_df = pd.concat([train_df, train_feat_df], axis=1)
    test_df = pd.concat([test_df, test_feat_df], axis=1)
    # SVD
    #svd = TruncatedSVD(n_components=2, random_state=2020)
    #all_svd_feats = pd.DataFrame(svd.fit_transform(all_feat_df), columns=[f"svd_{c}_{ix}" for ix in range(2)])
    #train_df = pd.concat([train_df, all_svd_feats.iloc[:len(train_df)]], axis=1)
    #test_df = pd.concat([test_df, all_svd_feats.iloc[len(train_df):].reset_index(drop=True)], axis=1)
    # NMF
    nmf = NMF(n_components=4, random_state=2020)
    all_nmf_feats = pd.DataFrame(nmf.fit_transform(all_feat_df), columns=[f"nmf_{c}_{ix}" for ix in range(4)])
    train_df = pd.concat([train_df, all_nmf_feats.iloc[:len(train_df)]], axis=1)
    test_df = pd.concat([test_df, all_nmf_feats.iloc[len(train_df):].reset_index(drop=True)], axis=1)    
    all_feats_dict[c] = all_feat_df

In [14]:
imp_feats = get_importance_cols(use_num=20)
use_multi_feats = [c for c in all_feats_dict.keys() if col_contain(c, imp_feats)]
multi_combi_list = list(itertools.combinations(use_multi_feats, 2))

In [15]:
for col_a, col_b in tqdm(multi_combi_list):
    all_feats1 = all_feats_dict[col_a]
    all_feats2 = all_feats_dict[col_b]
    #all_feats3 = all_feats_dict[col_c]
    # all_feats = pd.concat([all_feats1, all_feats2, all_feats3], axis=1)
    all_feats = pd.concat([all_feats1, all_feats2], axis=1)
    # nmf
    n_comp = 10
    nmf = NMF(n_components=n_comp, random_state=2020)
    all_nmf_feats = pd.DataFrame(
        nmf.fit_transform(all_feats),
        columns=[f"nmf_multi_{col_a}__{col_b}_{ix}" for ix in range(n_comp)]
        # columns=[f"svd_multi_{col_a}__{col_b}__{col_c}_{ix}" for ix in range(n_comp)]
    )
    train_df = pd.concat([train_df, all_nmf_feats.iloc[:len(train_df)]], axis=1)
    test_df = pd.concat([test_df, all_nmf_feats.iloc[len(train_df):].reset_index(drop=True)], axis=1)
    # svd
    """
    n_comp = 4
    svd = TruncatedSVD(n_components=n_comp, random_state=2020)
    all_svd_feats = pd.DataFrame(
        svd.fit_transform(all_feats),
        columns=[f"svd_multi_{col_a}__{col_b}_{ix}" for ix in range(n_comp)]
        # columns=[f"svd_multi_{col_a}__{col_b}__{col_c}_{ix}" for ix in range(n_comp)]
    )
    train_df = pd.concat([train_df, all_svd_feats.iloc[:len(train_df)]], axis=1)
    test_df = pd.concat([test_df, all_svd_feats.iloc[len(train_df):].reset_index(drop=True)], axis=1)
    """

### Simple-category encoding

In [16]:
for c in cat_cols:
    train_df[c], uniques = pd.factorize(train_df[c], sort=True)
    test_df[c] = uniques.get_indexer(test_df[c])

### Reduce data memory

In [17]:
for c in nume_cols:
    if train_df[c].nunique() > 1000:
        train_df[c] = train_df[c].astype(np.float32)
        test_df[c] = test_df[c].astype(np.float32)
    else:
        train_df[c] = train_df[c].astype(np.float16)
        test_df[c] = test_df[c].astype(np.float16)

In [18]:
for c in cat_cols:
    if train_df[c].max() > 32767:
        train_df[c] = train_df[c].astype(np.int32)
        test_df[c] = test_df[c].astype(np.int32)
    else:
        train_df[c] = train_df[c].astype(np.int16)
        test_df[c] = test_df[c].astype(np.int16)            

In [19]:
train_df.shape, test_df.shape

((33857, 649), (11259, 648))

### Change Category Name

In [20]:
train_df.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in train_df.columns]
test_df.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in test_df.columns]

### Aggregation

In [21]:
imp_feats = get_importance_cols(use_num=50)
imp_cat_cols = [c for c in imp_feats if c in cat_cols or c[:4] == "ohe_"]
imp_nume_cols = [c for c in imp_feats if c in nume_cols or c[:4] == "ohe_"]

In [22]:
len(imp_cat_cols), len(imp_nume_cols)

(32, 15)

In [23]:
all_df = pd.concat([train_df, test_df], axis=0, ignore_index=True)

In [24]:
for cat_col in tqdm(imp_cat_cols):
    for nume_col in imp_nume_cols:
        # one-hotは同じカテゴリの場合がある
        if cat_col == nume_col:
            continue
        all_df[f"agg_mean_{cat_col}_{nume_col}"] = \
            all_df.groupby(cat_col)[nume_col].transform("mean").astype(np.float32)
        all_df[f"agg_std_{cat_col}_{nume_col}"] = \
            all_df.groupby(cat_col)[nume_col].transform("std").astype(np.float32)
        all_df[f"agg_max_{cat_col}_{nume_col}"] = \
            all_df.groupby(cat_col)[nume_col].transform("max").astype(np.float32)
        all_df[f"agg_min_{cat_col}_{nume_col}"] = \
            all_df.groupby(cat_col)[nume_col].transform("min").astype(np.float32)
        all_df[f"diff_agg_mean_{cat_col}_{nume_col}"] = \
            all_df[nume_col] - all_df[f"agg_mean_{cat_col}_{nume_col}"]
        all_df[f"rel_agg_mean_{cat_col}_{nume_col}"] = \
            all_df[nume_col] / (1 + all_df[f"agg_mean_{cat_col}_{nume_col}"])
train_df = all_df.iloc[:len(train_df)].reset_index(drop=True)
test_df = all_df.iloc[len(train_df):].reset_index(drop=True)
del all_df
gc.collect()

58

In [25]:
use_cols = [c for c in train_df.columns if c not in multi_cat_cols + ["Salary", "No"]]

In [26]:
folds = KFold(n_splits=5, random_state=2020, shuffle=True)
fold_idx_list = [(trn_idx, val_idx) for trn_idx, val_idx in folds.split(train_df, train_df["Salary"])]

### Target Encoding

In [27]:
@feature_cache(reset=False)
def target_encoding(train_df, test_df):
    te_cols = [c for c in train_df.columns if c in cat_cols]
    for c in tqdm(te_cols):
        new_col = "te_" + c
        train_df[new_col] = 0
        test_df[new_col] = 0
        for trn_idx, val_idx in fold_idx_list:
            mean_val = train_df.loc[trn_idx].groupby(c)["Salary"].mean().astype(np.float32)
            train_df.loc[val_idx, new_col] = train_df.loc[val_idx, c].map(mean_val)
            test_df.loc[:, new_col] += test_df.loc[:, c].map(mean_val) / len(fold_idx_list)
        train_df[new_col] = train_df[new_col].astype(np.float32)
        test_df[new_col] = test_df[new_col].astype(np.float32)
    return train_df, test_df

In [28]:
train_df, test_df = target_encoding(train_df, test_df)

In [29]:
@feature_cache(reset=False)
def multiple_target_encoding(train_df, test_df):
     # multiple target encoding
    multi_te_cols = [c for c in train_df.columns if c in cat_cols or c[:4] == "ohe_"]
    imp_feats = get_importance_cols(use_num=30)
    multi_te_cols = [c for c in multi_te_cols if c in imp_feats]
    combi_multi_te_cols = list(itertools.combinations(multi_te_cols, 2))   

    for col_a, col_b in tqdm(combi_multi_te_cols):
        new_col = "te_" + col_a + "__" + col_b
        train_df[new_col] = 0
        test_df[new_col] = 0
        train_df["tmp"] = train_df[col_a].fillna("").astype(str) + train_df[col_b].fillna("").astype(str)
        test_df["tmp"] = test_df[col_a].fillna("").astype(str) + test_df[col_b].fillna("").astype(str)
        for trn_idx, val_idx in fold_idx_list:
            mean_val = train_df.loc[trn_idx].groupby("tmp")["Salary"].mean().astype(np.float32)
            train_df.loc[val_idx, new_col] = train_df.loc[val_idx, "tmp"].map(mean_val)
            test_df.loc[:, new_col] += test_df.loc[:, "tmp"].map(mean_val) / len(fold_idx_list)
        train_df[new_col] = train_df[new_col].astype(np.float32)
        test_df[new_col] = test_df[new_col].astype(np.float32)
        del train_df["tmp"], test_df["tmp"]
        gc.collect()
    return train_df, test_df

In [30]:
train_df, test_df = multiple_target_encoding(train_df, test_df)

In [31]:
use_cols = [c for c in train_df.columns if c not in multi_cat_cols + ["Salary", "No"]]
print(len(use_cols))

3715


In [32]:
#import random
#random.seed(2020)
#random.shuffle(use_cols)

## Training

In [33]:
lgb_params = {
            'objective': 'regression',
            "metric": "rmse",
            "verbosity": -1,
            "boosting": "gbdt",
            'learning_rate': 0.05,
            'num_leaves': 64,
            'min_data_in_leaf': 80, 
            'max_depth': 4,
            "bagging_freq": 5,
            "bagging_fraction": 0.8,
            "lambda_l1": 0.5,
            "lambda_l2": 0.5,
            "feature_fraction": 0.1,
            "seed": 2020,
            "num_threads": -1,
            "max_bins": 30
}

In [34]:
importances = pd.DataFrame()
oof_preds = np.zeros(len(train_df))
models = []

for fold_i, (trn_idx, val_idx) in enumerate(fold_idx_list):
    print(f"Fold {fold_i+1}")
    train_dataset = lgb.Dataset(
        train_df.loc[trn_idx, use_cols],
        label = train_df.loc[trn_idx, "Salary"]
    )
    valid_dataset = lgb.Dataset(
        train_df.loc[val_idx, use_cols],
        label = train_df.loc[val_idx, "Salary"]
    )
    model = lgb.train(
                lgb_params,
                train_dataset,
                3000,
                valid_sets = [train_dataset, valid_dataset],
                verbose_eval=100,
                early_stopping_rounds = 100,
                #feval = eval_f1,
                #callbacks = [log_callback],
    )
    imp_df = pd.DataFrame()
    imp_df['feature'] = use_cols
    imp_df['gain'] = model.feature_importance(importance_type="gain")
    importances = pd.concat([importances, imp_df], axis=0, sort=False)
    
    oof_preds[val_idx] = model.predict(train_df.loc[val_idx, use_cols])
    models.append(model)

Fold 1
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 20632.9	valid_1's rmse: 21568.9
[200]	training's rmse: 19471.7	valid_1's rmse: 20993.2
[300]	training's rmse: 18845	valid_1's rmse: 20813
[400]	training's rmse: 18337.4	valid_1's rmse: 20716.5
[500]	training's rmse: 17896.8	valid_1's rmse: 20648.8
[600]	training's rmse: 17528.7	valid_1's rmse: 20611.1
[700]	training's rmse: 17202.8	valid_1's rmse: 20576
[800]	training's rmse: 16870.4	valid_1's rmse: 20544.6
[900]	training's rmse: 16564.9	valid_1's rmse: 20533.6
[1000]	training's rmse: 16299.2	valid_1's rmse: 20510.7
[1100]	training's rmse: 16028.1	valid_1's rmse: 20512.8
Early stopping, best iteration is:
[1066]	training's rmse: 16121.2	valid_1's rmse: 20503
Fold 2
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 20676.3	valid_1's rmse: 21623.9
[200]	training's rmse: 19536.3	valid_1's rmse: 20962.8
[300]	training's rmse: 18877.4	valid_1's rmse: 20713.8
[400]	t

In [35]:
oof_score = np.sqrt(mean_squared_error(train_df["Salary"], oof_preds))
oof_score
# 20286.360358084217

20252.13076209123

In [36]:
importances.groupby("feature")["gain"].mean().sort_values(ascending=False).reset_index().iloc[:300]

,feature,gain
0,te_Country__CareerSatisfaction,2.059933e+13
1,te_Country__Age,1.733009e+13
2,te_Country__ohe_DevType_Student,1.715378e+13
3,te_Country__ohe_Methodology_Agile,1.547402e+13
4,te_YearsCodingProf__Currency,1.345133e+13
5,te_Country__Dependents,1.304710e+13
6,te_Country__FormalEducation,1.145838e+13
7,te_Country__SalaryType,1.094706e+13
8,te_Country__YearsCoding,1.031458e+13
9,te_YearsCodingProf__MilitaryUS,9.609372e+12


## Prediction

In [37]:
test_pred = np.zeros(len(test_df))

for model in models:
    test_pred += model.predict(test_df[use_cols]) / len(models)

In [38]:
test_pred

array([ 67645.43718659,  91251.64214448,  84434.28028299, ...,
        87282.93162139,  81075.80900633, 118959.49155246])

In [39]:
sub_df = pd.read_csv("../input/submit.csv")

In [40]:
sub_df["Salary"] = test_pred

In [42]:
sub_df.to_csv("../predict/016_svd_20252.csv", index=False)